In [3]:
import pyspark
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('2_frame').getOrCreate()
spark

24/03/14 06:07:43 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


## Pyspark DataFrame

In [35]:
!wget https://people.sc.fsu.edu/~jburkardt/data/csv/taxables.csv

--2024-03-14 06:22:09--  https://people.sc.fsu.edu/~jburkardt/data/csv/taxables.csv
Resolving people.sc.fsu.edu (people.sc.fsu.edu)... 144.174.0.22
Connecting to people.sc.fsu.edu (people.sc.fsu.edu)|144.174.0.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 611 [text/csv]
Saving to: ‘taxables.csv’

taxables.csv        100%[===================>]     611  --.-KB/s    in 0s      

2024-03-14 06:22:11 (83.7 MB/s) - ‘taxables.csv’ saved [611/611]



In [37]:
df = spark.read.format('csv').option('header', 'true').load(
    'taxables.csv', inferSchema=True
)

#or 
df = spark.read.csv("taxables.csv", header=True, inferSchema=True)

In [38]:
df.show()

+-----+--------------------+--------------------+------+--------+
|Index|              "Item"|              "Cost"| "Tax"| "Total"|
+-----+--------------------+--------------------+------+--------+
|  1.0| "Fruit of the Lo...|                7.97|   0.6|    8.57|
|  2.0| "Rawlings Little...|                2.97|  0.22|    3.19|
|  3.0| "Secret Antipers...|                1.29|   0.1|    1.39|
|  4.0|      "Deadpool DVD"|                 ...|  1.12|   16.08|
|  5.0| "Maxwell House C...|                7.28|  0.55|    7.83|
|  6.0| "Banana Boat Sun...|               8 oz"|  6.68|     0.5|
|  7.0|         "Wrench Set|          18 pieces"|  10.0|    0.75|
|  8.0|            "M and M|              42 oz"|  8.98|    0.67|
|  9.0| "Bertoli Alfredo...|                2.12|  0.16|    2.28|
| 10.0|   "Large Paperclips|           10 boxes"|  6.19|    0.46|
+-----+--------------------+--------------------+------+--------+



## Schema

In [39]:
df.printSchema()

root
 |-- Index: double (nullable = true)
 |--  "Item": string (nullable = true)
 |--  "Cost": string (nullable = true)
 |--  "Tax": double (nullable = true)
 |--  "Total": double (nullable = true)



In [40]:
type(df)

pyspark.sql.dataframe.DataFrame

## Getting columns

In [49]:
columns = df.columns
columns

['Index', ' "Item"', ' "Cost"', ' "Tax"', ' "Total"']

In [50]:
df.select(columns[3]).show(5)

+------+
| "Tax"|
+------+
|   0.6|
|  0.22|
|   0.1|
|  1.12|
|  0.55|
+------+
only showing top 5 rows



In [53]:
df.select(columns[1:3]).show()

+--------------------+--------------------+
|              "Item"|              "Cost"|
+--------------------+--------------------+
| "Fruit of the Lo...|                7.97|
| "Rawlings Little...|                2.97|
| "Secret Antipers...|                1.29|
|      "Deadpool DVD"|                 ...|
| "Maxwell House C...|                7.28|
| "Banana Boat Sun...|               8 oz"|
|         "Wrench Set|          18 pieces"|
|            "M and M|              42 oz"|
| "Bertoli Alfredo...|                2.12|
|   "Large Paperclips|           10 boxes"|
+--------------------+--------------------+



## Describe dataframe

In [55]:
df.describe().show()

24/03/14 06:25:57 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+--------------------+--------------------+-----------------+-----------------+
|summary|             Index|              "Item"|              "Cost"|            "Tax"|          "Total"|
+-------+------------------+--------------------+--------------------+-----------------+-----------------+
|  count|                10|                  10|                  10|               10|               10|
|   mean|               5.5|                NULL|   6.098333333333333|             3.46|            4.172|
| stddev|3.0276503540974917|                NULL|   5.141250496393526|4.025802887485786|5.142891102180648|
|    min|               1.0| "Banana Boat Sun...|                 ...|              0.1|             0.46|
|    max|              10.0|         "Wrench Set|               8 oz"|             10.0|            16.08|
+-------+------------------+--------------------+--------------------+-----------------+-----------------+



In [56]:
df.dtypes

[('Index', 'double'),
 (' "Item"', 'string'),
 (' "Cost"', 'string'),
 (' "Tax"', 'double'),
 (' "Total"', 'double')]

## Adding new column

In [58]:
df[columns[3]]

Column<' "Tax"'>

In [63]:
df = df.withColumn('New Tax', df[columns[3]] + 10)

In [65]:
df.show()

+-----+--------------------+--------------------+------+--------+------------------+
|Index|              "Item"|              "Cost"| "Tax"| "Total"|           New Tax|
+-----+--------------------+--------------------+------+--------+------------------+
|  1.0| "Fruit of the Lo...|                7.97|   0.6|    8.57|              10.6|
|  2.0| "Rawlings Little...|                2.97|  0.22|    3.19|             10.22|
|  3.0| "Secret Antipers...|                1.29|   0.1|    1.39|              10.1|
|  4.0|      "Deadpool DVD"|                 ...|  1.12|   16.08|11.120000000000001|
|  5.0| "Maxwell House C...|                7.28|  0.55|    7.83|             10.55|
|  6.0| "Banana Boat Sun...|               8 oz"|  6.68|     0.5|             16.68|
|  7.0|         "Wrench Set|          18 pieces"|  10.0|    0.75|              20.0|
|  8.0|            "M and M|              42 oz"|  8.98|    0.67|             18.98|
|  9.0| "Bertoli Alfredo...|                2.12|  0.16|    2.28|

## Removing columns

In [67]:
df = df.drop(columns[3])

In [69]:
df.show()

+-----+--------------------+--------------------+--------+------------------+
|Index|              "Item"|              "Cost"| "Total"|           New Tax|
+-----+--------------------+--------------------+--------+------------------+
|  1.0| "Fruit of the Lo...|                7.97|    8.57|              10.6|
|  2.0| "Rawlings Little...|                2.97|    3.19|             10.22|
|  3.0| "Secret Antipers...|                1.29|    1.39|              10.1|
|  4.0|      "Deadpool DVD"|                 ...|   16.08|11.120000000000001|
|  5.0| "Maxwell House C...|                7.28|    7.83|             10.55|
|  6.0| "Banana Boat Sun...|               8 oz"|     0.5|             16.68|
|  7.0|         "Wrench Set|          18 pieces"|    0.75|              20.0|
|  8.0|            "M and M|              42 oz"|    0.67|             18.98|
|  9.0| "Bertoli Alfredo...|                2.12|    2.28|             10.16|
| 10.0|   "Large Paperclips|           10 boxes"|    0.46|      

## Rename column

In [70]:
df = df.withColumnRenamed('New Tax', 'Tax')

In [71]:
df.show(4)

+-----+--------------------+--------------------+--------+------------------+
|Index|              "Item"|              "Cost"| "Total"|               Tax|
+-----+--------------------+--------------------+--------+------------------+
|  1.0| "Fruit of the Lo...|                7.97|    8.57|              10.6|
|  2.0| "Rawlings Little...|                2.97|    3.19|             10.22|
|  3.0| "Secret Antipers...|                1.29|    1.39|              10.1|
|  4.0|      "Deadpool DVD"|                 ...|   16.08|11.120000000000001|
+-----+--------------------+--------------------+--------+------------------+
only showing top 4 rows

